In [26]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///../mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [27]:
client.search_experiments()

[<Experiment: artifact_location='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/2', creation_time=1703051127573, experiment_id='2', last_update_time=1703051127573, lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1', creation_time=1702972736474, experiment_id='1', last_update_time=1702972736474, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [28]:
# client.create_experiment(name="my-cool-experiment")

In [33]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids="1",
    filter_string="metrics.rmse < 8.0",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [34]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 7aaff75c0f8248028684f331ecac6a9e, rmse: 6.3136
run id: e29ba9cc7042477a847786e3e1a97538, rmse: 7.7037
run id: ede9475e9aed43ccb06ee98bc872e17c, rmse: 7.7587


In [35]:
# Add models to registry with python api

import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [39]:
run_id = "e29ba9cc7042477a847786e3e1a97538"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '3' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1703145956166, current_stage='None', description=None, last_updated_timestamp=1703145956166, name='nyc-taxi-regressor', run_id='e29ba9cc7042477a847786e3e1a97538', run_link=None, source='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1/e29ba9cc7042477a847786e3e1a97538/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [40]:
# Transition model from one stage to another
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1703144139498, description='', last_updated_timestamp=1703145956166, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1703144139562, current_stage='Production', description='', last_updated_timestamp=1703144393734, name='nyc-taxi-regressor', run_id='ede9475e9aed43ccb06ee98bc872e17c', run_link='', source='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1/ede9475e9aed43ccb06ee98bc872e17c/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>,
  <ModelVersion: aliases=[], creation_timestamp=1703145688787, current_stage='Staging', description='', last_updated_timestamp=1703145699852, name='nyc-taxi-regressor', run_id='7aaff75c0f8248028684f331ecac6a9e', run_link='', source='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1/7aaff75c0f8248028684f331ecac6a9e/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=2>,
  <ModelV

In [41]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Production
version: 2, stage: Staging
version: 3, stage: None


/tmp/ipykernel_6207/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [42]:
model_version = 3
new_stage = "Staging"

# Stage a model
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_6207/3632994024.py:5: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1703145956166, current_stage='Staging', description=None, last_updated_timestamp=1703145976072, name='nyc-taxi-regressor', run_id='e29ba9cc7042477a847786e3e1a97538', run_link=None, source='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1/e29ba9cc7042477a847786e3e1a97538/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [43]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1703145956166, current_stage='Staging', description='The model version 3 was transitioned to Staging on 2023-12-21', last_updated_timestamp=1703145982195, name='nyc-taxi-regressor', run_id='e29ba9cc7042477a847786e3e1a97538', run_link=None, source='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1/e29ba9cc7042477a847786e3e1a97538/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>

## Comparing versions and selecting the new "Production" model
 The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.
 These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.

**Note:** The model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.

In [51]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    # Calculate duration
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    # Filter between 1 min and 60 mins
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']    
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [45]:
df = read_dataframe('../data/green_tripdata_2021-03.parquet')

In [46]:
model_name = "nyc-taxi-regressor"

In [47]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.') 

'/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/preprocessor'

In [48]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [49]:
X_test = preprocess(df, dv)

In [50]:
target = "duration"
y_test = df[target].values

In [52]:
%time test_model(name=model_name, stage="Production", X_test=X_test, y_test=y_test)

/home/pytholic/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


CPU times: user 399 ms, sys: 24.2 ms, total: 423 ms
Wall time: 462 ms


{'rmse': 7.720735231402423}

In [53]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

CPU times: user 34.8 ms, sys: 417 µs, total: 35.2 ms
Wall time: 34 ms


/home/pytholic/anaconda3/envs/exp-tracking/lib/python3.9/site-packages/mlflow/store/artifact/utils/models.py:32: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  latest = client.get_latest_versions(name, None if stage is None else [stage])


{'rmse': 7.687093496971111}

In [54]:
client.transition_model_version_stage(
    name=model_name,
    version=3,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_6207/684772386.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1703145956166, current_stage='Production', description='The model version 3 was transitioned to Staging on 2023-12-21', last_updated_timestamp=1703146205796, name='nyc-taxi-regressor', run_id='e29ba9cc7042477a847786e3e1a97538', run_link=None, source='/home/pytholic/mlops-zoomcamp-pytholic/02-experiment-tracking/mlruns/1/e29ba9cc7042477a847786e3e1a97538/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=3>